In [ ]:
from openai import OpenAI
client = OpenAI()

javaland_embedding = client.embeddings.create(input = ["Javaland"], model="text-embedding-3-small").data[0].embedding

print(javaland_embedding[:10])

In [ ]:
import numpy

javaland_embedding = numpy.array(client.embeddings.create(input = ["Javaland"], model="text-embedding-3-small").data[0].embedding)

java_embedding = numpy.array(client.embeddings.create(input = ["Java"], model="text-embedding-3-small").data[0].embedding)

python_embedding = numpy.array(client.embeddings.create(input = ["Python"], model="text-embedding-3-small").data[0].embedding)

java_dist = numpy.linalg.norm(javaland_embedding-java_embedding)

print("Java to Javaland distance: " + str(java_dist))

python_dist = numpy.linalg.norm(javaland_embedding-python_embedding)

print("Python to Javaland distance: " + str(python_dist))


In [ ]:
import json

with open('./javaland-details.json', 'r') as file:
    data = json.load(file)

print(len(data))
sessions = []
for session in data:
    if "textSearch" in session:
        sessions.append(session)

print(len(sessions))

In [ ]:
import html2text

h = html2text.HTML2Text()
h.ignore_links

head = sessions[:3]

for session in head:
    print(session["textSearch"])
    print("---------")
    print(h.handle(session["textSearch"]))
    print("+++++++++")



In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

with open('./javaland-agenda.json', 'r') as file:
    sessions = json.load(file)

texts = []
metadatas = []
for session in sessions:
    title = f"Titel: ${session["title"]} /n"
    speakerstr = f"Speaker: ${session["speaker"]} /n"
    main_focus = f"Main Focus: ${session["main_focus"]} /n" if "main_focus" in session else ""
    text_description = h.handle(session["text"])

    text = title + speakerstr + main_focus + text_description
    texts.append(text)

db = Chroma.from_texts(
        texts,
        embedding=OpenAIEmbeddings(),
        metadatas=sessions,
        persist_directory="./chroma"
    )




In [ ]:
db2 = Chroma(persist_directory="./chroma", embedding_function=OpenAIEmbeddings())

query = "Which sessions are about AI?"

docs = db2.similarity_search(query)

for doc in docs:
    print(doc)




In [ ]:
docs = db2.max_marginal_relevance_search(query)

for doc in docs:
    print(doc.metadata['title'])




In [ ]:
docs = db2.similarity_search_with_score(query)

for doc, score in docs:
    print(doc.metadata['title'] + "Score:" + str(score))


In [ ]:
docs = db2.similarity_search_with_score(query, filter={"main_focus":"Data_Streaming_KI"})

for doc, score in docs:
    print(doc.metadata['title'] + "Score:" + str(score))